In [1]:
!pip install -q polars


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import polars as pl
import os
from datetime import datetime

min_date = []
max_date = []
files = os.listdir("result")
len(files)

4539

In [3]:


for d in files:
    df = pl.read_csv(f"result/{d}").sort("date")
    if df.height > 2 and df[-1, "date"] == df[-2, "date"]:
        df = df[:-1]
    df = df.with_columns(
        pl.col("date").str.to_datetime(format="%Y-%m-%d %H:%M:%S").dt.truncate("1m")
    )

    temp = df["date"].value_counts().sort("date").filter(pl.col("count") > 1)
    if temp.shape[0]:
        print(d, "has duplicates")
        print(temp)

    df = df.unique("date", keep="last")
    min_date.append(df["date"].dt.min())
    max_date.append(df["date"].dt.max())


min_date = pl.Series("min_date", min_date).dt.min()
max_date = pl.Series("max_date", max_date).dt.max()

print(min_date, max_date)

debridge.csv has duplicates
shape: (14, 2)
┌─────────────────────┬───────┐
│ date                ┆ count │
│ ---                 ┆ ---   │
│ datetime[μs]        ┆ u32   │
╞═════════════════════╪═══════╡
│ 2022-02-25 21:00:00 ┆ 2     │
│ 2022-02-26 12:00:00 ┆ 2     │
│ 2022-02-26 17:59:00 ┆ 2     │
│ 2022-02-27 01:00:00 ┆ 2     │
│ 2022-02-28 00:02:00 ┆ 2     │
│ …                   ┆ …     │
│ 2022-07-06 20:59:00 ┆ 2     │
│ 2022-07-12 21:00:00 ┆ 2     │
│ 2022-07-13 05:03:00 ┆ 2     │
│ 2022-10-20 21:19:00 ┆ 2     │
│ 2022-10-21 20:40:00 ┆ 2     │
└─────────────────────┴───────┘
fantom.csv has duplicates
shape: (6, 2)
┌─────────────────────┬───────┐
│ date                ┆ count │
│ ---                 ┆ ---   │
│ datetime[μs]        ┆ u32   │
╞═════════════════════╪═══════╡
│ 2021-03-28 10:02:00 ┆ 2     │
│ 2021-05-22 21:40:00 ┆ 2     │
│ 2021-05-22 21:42:00 ┆ 2     │
│ 2021-05-22 23:36:00 ┆ 2     │
│ 2021-05-23 01:06:00 ┆ 2     │
│ 2021-06-26 05:57:00 ┆ 2     │
└────────────────────

In [4]:
from datetime import datetime, timedelta


date_list = []
current_date = min_date

while current_date <= max_date:
    date_list.append(current_date)
    current_date += timedelta(minutes=1)


# Polars DataFrame으로 변환
result_raw = pl.DataFrame({
    "date": [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in date_list]
}).with_columns(
    pl.col("date").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S")
)

result_raw


date
datetime[μs]
2018-04-27 09:00:00
2018-04-27 09:01:00
2018-04-27 09:02:00
2018-04-27 09:03:00
2018-04-27 09:04:00
…
2024-08-18 16:35:00
2024-08-18 16:36:00
2024-08-18 16:37:00


In [6]:
STEP = 1600
os.makedirs(f"result{STEP}", exist_ok=True)
for start in range(0, len(files), STEP):
    result = result_raw
    for d in files[start:start+STEP]:
        df = pl.read_csv(f"result/{d}").sort("date")
        if df.height > 2 and df[-1, "date"] == df[-2, "date"]:
            df = df[:-1]
        df = df.with_columns(
            pl.col("date").str.to_datetime(format="%Y-%m-%d %H:%M:%S").dt.truncate("1m")
        ).unique("date", keep="last")

        result = result.join(df, on="date", how="left")
    result.write_csv(f"result{STEP}/result{start}.csv")